In [51]:
#Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

import tensorflow as tf
print("TensorFlow version:", tf.__version__) #Printing tensorflow version to ensure library correctly installed

TensorFlow version: 2.16.1


In [52]:
df_medical_data = pd.read_csv('./Medicaldataset.csv') #Initialise dataframe from Medicaldataset.csv
df_medical_data.head(10) #Printing the first 10 rows of the dataset (dataset has 1319 rows)

,Age,Gender,Heart rate,Systolic blood pressure,Diastolic blood pressure,Blood sugar,CK-MB,Troponin,Result
0,63,1,66,160,83,160.0,1.80,0.012,negative
1,20,1,94,98,46,296.0,6.75,1.060,positive
2,56,1,64,160,77,270.0,1.99,0.003,negative
3,66,1,70,120,55,270.0,13.87,0.122,positive
4,54,1,64,112,65,300.0,1.08,0.003,negative
5,52,0,61,112,58,87.0,1.83,0.004,negative
6,38,0,40,179,68,102.0,0.71,0.003,negative
7,61,1,60,214,82,87.0,300.00,2.370,positive
8,49,0,60,154,81,135.0,2.35,0.004,negative
9,65,1,61,160,95,100.0,2.84,0.011,negative


In [53]:
df_medical_data.Result.value_counts()

Result
positive    810
negative    509
Name: count, dtype: int64

Based of the available labels in the 'Result' column we can see that on the 'positive' and 'negative' labels are presents hence we are dealing with a binary classification model.

In [54]:
df_medical_data.shape


(1319, 9)

In [55]:
def column_min_max(df, column):
    min_value = df[column].min()
    max_value = df[column].max()
    print(f"'{column}' range is [{min_value} , {max_value}]")

columns = ['Age', 'Gender', 'Heart rate', 'Systolic blood pressure', 'Diastolic blood pressure', 'Blood sugar', 'CK-MB', 'Troponin']

for column in columns:
    column_min_max(df_medical_data, column)

'Age' range is [14 , 103]
'Gender' range is [0 , 1]
'Heart rate' range is [20 , 1111]
'Systolic blood pressure' range is [42 , 223]
'Diastolic blood pressure' range is [38 , 154]
'Blood sugar' range is [35.0 , 541.0]
'CK-MB' range is [0.321 , 300.0]
'Troponin' range is [0.001 , 10.3]


### Normalisation of data:

##### 'Result' converted to 0.0 for negative and 1.0 for positive.

##### All other columns aside from 'Gender" are scaled to between 0.0 and 1.0

In [57]:
df_medical_data['Result'] = df_medical_data['Result'].replace({'negative': 0, 'positive': 1})

scaler = MinMaxScaler()
df_medical_data[['Age', 'Gender', 'Heart rate', 'Systolic blood pressure', 'Diastolic blood pressure', 'Blood sugar', 'CK-MB', 'Troponin', 'Result']] = scaler.fit_transform(df_medical_data[['Age', 'Gender', 'Heart rate', 'Systolic blood pressure', 'Diastolic blood pressure', 'Blood sugar', 'CK-MB', 'Troponin', 'Result']])

df_medical_data.head(10)

,Age,Gender,Heart rate,Systolic blood pressure,Diastolic blood pressure,Blood sugar,CK-MB,Troponin,Result
0,0.550562,1.0,0.042163,0.651934,0.387931,0.247036,0.004935,0.001068,0.0
1,0.067416,1.0,0.067828,0.309392,0.068966,0.515810,0.021453,0.102826,1.0
2,0.471910,1.0,0.040330,0.651934,0.336207,0.464427,0.005569,0.000194,0.0
3,0.584270,1.0,0.045830,0.430939,0.146552,0.464427,0.045212,0.011749,1.0
4,0.449438,1.0,0.040330,0.386740,0.232759,0.523715,0.002533,0.000194,0.0
5,0.426966,0.0,0.037580,0.386740,0.172414,0.102767,0.005035,0.000291,0.0
6,0.269663,0.0,0.018332,0.756906,0.258621,0.132411,0.001298,0.000194,0.0
7,0.528090,1.0,0.036664,0.950276,0.379310,0.102767,1.000000,0.230022,1.0
8,0.393258,0.0,0.036664,0.618785,0.370690,0.197628,0.006771,0.000291,0.0
9,0.573034,1.0,0.037580,0.651934,0.491379,0.128458,0.008406,0.000971,0.0


##### Begin splitting data and forming training, validation and test sets

In [74]:
#Split dataframe into input and output sets. 
X = df_medical_data.drop('Result', axis=1)
y = df_medical_data['Result']


# random_state=42 makes sures the RNG used by train_test_split is initialized to the same state everytime and ensures that the data is always split the same way
# Ratio split is Train:Validation:Test = 60:20:20
X_temp, X_test, y_temp, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.25, random_state=42)

print(X_train.shape, y_train.shape, X_val.shape, y_val.shape, X_test.shape, y_test.shape) #Ensuring the size of the input and output for each set match

(791, 8) (791,) (264, 8) (264,) (264, 8) (264,)
